# In-context Learning (Few-shot prompting): DeepSeek

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import openai
from openai import OpenAI
import os

### Prepare the data

In [3]:
prompting_data = pd.read_csv('context_data_48.csv')
testing_data = pd.read_csv('testing_data_64.csv')

### Create Prompting

In [6]:
def create_few_shot_prompt(text, prompting_data, N=0):
    """
    Creates a prompt for few-shot learning by including N examples from prompting_data.
    If N=0, it performs zero-shot learning.

    Args:
        text (str): The input text to analyze.
        prompting_data (list of dict): A list of dictionaries containing 'post' and 'mbti_type'.
        N (int): The number of examples to include in the prompt. If 0, zero-shot learning is used.

    Returns:
        str: The constructed prompt.
    """
    # Base instruction for the task
    prompt = """
    Analyze the following text and predict the most likely MBTI (Myers-Briggs Type Indicator) personality type. 
    Provide a detailed reasoning for your prediction.
    """

    # Add few-shot examples if N > 0
    if N > 0:
        prompt += "\n\nHere are some examples to guide your prediction:\n"
        for i, row in prompting_data.head(N).iterrows():  # Take the first N rows
            prompt += f"""
            Example {i + 1}:
            Text: {row['tweet']}
            MBTI Type: {row['type']}
            """
        prompt += "\nNow, analyze the following text:\n"

    # Add the input text to analyze
    prompt += f"""
    Text: {text}

    MBTI Type Prediction:
    Reasoning:
    """

    return prompt

### Predict MBTI

In [ ]:
client = OpenAI(api_key="***", base_url="https://api.deepseek.com")

def predict_mbti(text, prompting_data, N=0):
    # Create the prompt
    prompt = create_few_shot_prompt(text, prompting_data, N)

    messages = [{"role": "user", "content": prompt}]

    response = client.chat.completions.create(
        model="deepseek-reasoner",
        messages=messages
    )

    # Extract the response content
    result = response.choices[0].message.content

    return result  

In [14]:
import json

N_samples = 48
results = []

for idx, row in testing_data.iterrows():
    text = row['tweet']
    actual_type = row['type']

    response = predict_mbti(text, prompting_data, N_samples)

    print(f"Text: {text}\n")
    print(f"Response: \n{response}\n")
    print(f"Actual type: {actual_type}\n")
    print(f"Processed row {idx + 1}/{len(testing_data)}")
    print("----------------------------------------------")

    results.append({
        "text:" : text,
        "response" : response,
        "actual_type": actual_type
    })

with open('Deepseek_N48.json', 'w') as f:
    json.dump(results, f, indent=4)

print("MBTI predictions saved to 'Deepseek_N48.json'.")

Text: Yes and due to it being shapes and objects that means our process of analyzing is not so much bound by languages

Response: 
**MBTI Type Prediction: INTP**

**Reasoning:**  
The text emphasizes abstract analysis ("shapes and objects") and a process unconstrained by linguistic structures. This reflects **Intuition (N)** over Sensing (S), as the focus is on conceptual patterns rather than concrete details. The logical, systems-oriented approach to analysis ("process of analyzing") points to **Thinking (T)** over Feeling (F).  

The phrase "not so much bound by languages" suggests a preference for non-traditional, flexible frameworks, aligning with **Introverted Thinking (Ti)**, which seeks internal logical coherence, and **Extraverted Intuition (Ne)**, which explores abstract possibilities. Ti (dominant in INTPs) prioritizes subjective logic and deconstructing systems, while Ne (auxiliary) drives curiosity about alternative methods of understanding.  

The lack of emphasis on socia

In [ ]:
with open('Deepseek_N48.json', 'w') as f:
    json.dump(results, f, indent=4)

print("MBTI predictions saved to 'Deepseek_N48.json'.")